# Скачивание новостного корпуса

Этот набор упражнений должен помочь вам научиться:

- выгружать интересующие вас тексты из Интернета
- сохранять их в формате, удобном для обработки
- выполнять их разметку

[Ссылка](https://colab.research.google.com/drive/1M5a8UUVcqW_wjNTknl2hjsHYsNVJEnPn?usp=sharing) на тетрадку в Google Colab

Возможно, если вы решите на практике применить то же самое к своему материалу, эта тетрадка не настолько подробной, как хотелось бы. В этом случае можно и нужно обратиться с вопросами в личные сообщения.

In [1]:
#Скачиваем пакеты, которых нет в стандартной библиотеке Python
!pip install fake-useragent
!pip install tqdm

Импортируем нужные для работы модули

In [4]:
#Собственно выкачивание файлов из Интернета
import requests

#Работа с таблицами — понадобится, чтобы хранить наш корпус
import pandas as pd

#Выполнение быстрых математических вычислений — пригодится, потому что иначе
#корпус с большим количеством текстов может долго обрабатываться
import numpy as np

# Модуль для работы с HTML-файлами
from bs4 import BeautifulSoup

# Модуль для работы с регулярными выражениями
import re

#Функция, который измеряет время исполнения фрагмента программы — без неё
#в принципе тоже всё будет работать
from tqdm.auto import tqdm

#Поможет замаскироваться под обычного пользователя в Интернете
from fake_useragent import UserAgent

#Переменные, нужные, чтобы маскироваться под обычного пользователя Интернета
ua = UserAgent()
headers = {'User-Agent': ua.random}

## Сбор текстов в Интернете

Python позволяет достаточно простым образом скачивать страницы из интернета по ссылке. Например, ниже скачивается и записывается в переменную файл с предыдущего занятия.

In [5]:
akarenina_link = 'https://raw.githubusercontent.com/alekseyst/text_analysis_2023/main/Seminar_3/karenina.txt'
akarenina = requests.get(akarenina_link, headers=headers)
akarenina

<Response [200]>

In [6]:
#Как мы видим, результат запроса представляет собой какой-то объект.
#Чтобы извлечь его содержимое, нужно обратиться к атрибуту text.

akarenina.text[:400]

'\r\n Л.Н. Толстой. Анна Каренина \r\n\r\n Роман в восьми частях \r\n\r\n\t Мне отмщение, и Аз воздам  \r\n\t \r\n\r\n Часть первая \r\n\r\n I \r\n\r\n Все счастливые семьи похожи друг на друга, каждая несчастливая семья несчастлива по-своему. \r\n Все смешалось в доме Облонских. Жена узнала, что муж был в связи с бывшею в их доме француженкою-гувернанткой, и объявила мужу, что не может жить с ним в одном доме. Положение это '

##HTML

Если мы, однако, имеем дело не с txt-файлами, выложенными в Интернет, а с обычными страницами, приходится иметь дело с HTML.

In [7]:
#HTML-страница новости

news_cher_link = "https://cher-is.com/izvestnaya-nejroset-sostavila-obrazy-cherepovtsa-i-vologdy/"
html_cher_request_result = requests.get(news_cher_link, headers=headers)
html_cher = html_cher_request_result.text
html_cher[:800]

'<!DOCTYPE html>\n<html lang="ru-RU" itemscope itemtype="http://schema.org/WebSite" prefix="og: http://ogp.me/ns#">\n<head> <a href="https://metrika.yandex.ru/stat/?id=52125751&amp;from=informer" target="_blank" rel="nofollow"><img src="https://informer.yandex.ru/informer/52125751/3_1_FFFFFFFF_EFEFEFFF_0_pageviews" style="width:88px; height:31px; border:0;" alt="Яндекс.Метрика" title="Яндекс.Метрика: данные за сегодня (просмотры, визиты и уникальные посетители)" class="ym-advanced-informer" data-cid="52125751" data-lang="ru" /></a>   <script type="text/javascript"> (function(m,e,t,r,i,k,a){m[i]=m[i]||function(){(m[i].a=m[i].a||[]).push(arguments)}; m[i].l=1*new Date();k=e.createElement(t),a=e.getElementsByTagName(t)[0],k.async=1,k.src=r,a.parentNode.insertBefore(k,a)}) (window, document, "scr'

Конечно, страшно, но не очень.

Если хочется прямо разобраться с HTML, вот, например, [ссылка](https://www.w3schools.com/html/html_intro.asp). А так нам важно знать, что всякие элементы располагаются в нутри тегов, и выглядит это как-то так (это тэг \<p\>, который используется для разбивки страницы на абзацы — paragraph):

\<p\>This is a paragraph.\</p\>

Обычно достаточно уметь найти, в каком тэге находится нужная информация, и извлечь её. Для поиска нужно всего лишь нажать на то, что вы хотите извлечь со страницы правой клавишей — Inspect, и после этого вы попадёте в удивительный мир HTML. Но! Вас переведут ровно к тому элементу, который вы ткнули. После этого, поводив по экрану, на котором подсвечиваются блоки, соответствующие тэгам, можно наиболее точно найти то, что вас интересует.

Новость выкачанной нами страницы, включая заголовок и картинку, находится в тэге \<div class="blog-lg-box"\>Содержимое\</div\>.

Для извлечения можно использовать библиотеку BeautifulSoup, которую мы уже импортировали.

In [8]:
#Просим модуль проанализировать наш файл
html_cher_soup = BeautifulSoup(html_cher, 'html.parser')

#Ищем
body_cher_soup = html_cher_soup.find("div", class_="media-body")

#Тэг — первая переменная, класс — вторая
#Если хотите найти несколько тэгов с одинаковыми классами, используйте find_all
#Они будут выданы в виде списка объектов BeautifulSoup — из каждого придётся
#извлекать текст

body_cher_soup

<div class="media-body">
<div class="blog-lg-box">
<a class="img-responsive" href="https://cher-is.com/izvestnaya-nejroset-sostavila-obrazy-cherepovtsa-i-vologdy/" title="Известная нейросеть составила образы Череповца и Вологды">
<img alt="" class="img-responsive wp-post-image" height="408" loading="lazy" sizes="(max-width: 264px) 100vw, 264px" src="https://cher-is.com/wp-content/uploads/2023/02/cherepovets.png" srcset="https://cher-is.com/wp-content/uploads/2023/02/cherepovets.png 264w, https://cher-is.com/wp-content/uploads/2023/02/cherepovets-194x300.png 194w" width="264"/>
</a></div>
<div class="blog-post-lg">
<a href="https://cher-is.com/author/admin/"><img alt="" class="img-responsive img-circle avatar-40 photo" height="40" loading="lazy" src="https://secure.gravatar.com/avatar/12d14aad72f86a5c2271b4922c91b55f?s=40&amp;d=mm&amp;r=g" srcset="https://secure.gravatar.com/avatar/12d14aad72f86a5c2271b4922c91b55f?s=80&amp;d=mm&amp;r=g 2x" width="40"/></a>
Автор:<a href="https://cher-is

Уже похоже на новость! Если теперь удалить все оставшиеся тэги, которые нас явно не интересуют, можно получить собственно текст.

In [9]:
cher_news = re.sub('<[^<]+?>', '', body_cher_soup.text).strip()

#Как это работает? Регулярное выражение заменяет все (практически все) тэги на
#пустую строку. Метод .strip() удаляет лишние переносы строк и подобное по краям
#новости
#Осторожно! Если вы занимаетесь чем-то серьёзным и работаете с потенциально
#опасными сайтами, удалять тэги следует другим способом.

cher_news

'Автор:ЧИ\nMidjourney, нейросеть, Череповец\n\nИзвестная нейросеть составила образы Череповца и Вологды\nНейросеть Midjourney изобразила города России в образах, и получились герои для мрачных сказок.\nНа этот раз автор телеграм-канала «Нейросеть видит» сформировал запрос в Midjourney так, чтобы нейросеть показала российские города в человеческих обличьях. Например, две столицы, Москва и Санкт-Петербург, выглядят как модная пара. Девушка — в стильном кокошнике, а парень с уложенными бородой и усами.\nЧереповец изображён в виде монаха, а Вологда — знатной женщины.'

Чтобы избавиться от автора, названия картинки и остального, можно их удалить в полученном выше тексте или извлекать из текста новости только то, что содержится в тэгах \<p\>\</p\>.

In [10]:
cher_news_no_author = ''

#Приходится использовать цикл, потому что метод find_all выдаёт как результат
#не строки, а объекты модуля Beautiful Soup

for p in body_cher_soup.find_all("p"):
  cher_news_no_author += p.text.strip()

cher_news_no_author

'Нейросеть Midjourney изобразила города России в образах, и получились герои для мрачных сказок.На этот раз автор телеграм-канала «Нейросеть видит» сформировал запрос в Midjourney так, чтобы нейросеть показала российские города в человеческих обличьях. Например, две столицы, Москва и Санкт-Петербург, выглядят как модная пара. Девушка — в стильном кокошнике, а парень с уложенными бородой и усами.Череповец изображён в виде монаха, а Вологда — знатной женщины.'

## Одна новость — хорошо, но мало

Чтобы сделать корпус, выкачивания одной интернет-страницы, конечно, недостаточно. Хотелось бы выкачивать сразу много текстов — или в целом из Интернета, или хотя бы с одного сайта.

В целом, выкачать всю нужную информацию с сайта можно, найдя закономерность в том, как называются её страницы. Иногда это просто (например, каждая из новостей может быть просто пронумерована), иногда требуется больше изощрённости.

Рассмотрим сайт Панорамы. Ссылка на новость на этом сайте содержит просто транслитерированное название (https://panorama.pub/news/osibki-pri-vvode-pin-koda), поэтому мы не имеем к ней прямого доступа и можем только откуда-то извлечь с сайта.

На сайте Панорамы есть страница с новостями за весь день вот такого вида: https://panorama.pub/news/28-2-2024

Мы будем идти по этим страницам и выгружать ссылки со всеми новостями за каждый день.

In [11]:
#Определяем дату

from datetime import datetime, timedelta

In [26]:
#Генерируем даты за последние n=100 дней
#

dates = []
n = 100

date_list = [datetime.today() - timedelta(days=x) for x in range(n)]

In [27]:
#Преобразуем их в тот формат, который на странице новостей: день-месяц-год
#Запись с "f" перед началом строки позволяет сгенерировать строку в том виде,
#в котором нам удобно: в фигурные скобки {} можно подставить какое-то значение,
#например, переменную

date_list = [f"{day.day}-{day.month}-{day.year}" for day in date_list]

['28-2-2024',
 '27-2-2024',
 '26-2-2024',
 '25-2-2024',
 '24-2-2024',
 '23-2-2024',
 '22-2-2024',
 '21-2-2024',
 '20-2-2024',
 '19-2-2024']

In [28]:
#Вот последняя дата, на которую мы смотрим

date_list[-1]

'21-11-2023'

In [31]:
#В этой огромной ячейке мы, собственно, выкачиваем наши новости
#Мы сразу складываем их в списки, которые потом будет удобно преобразовать в таблицу с метаинформацией
all_titles = []
all_dates = []
all_links = []
all_texts = []
all_spheres = []

#Начинаем цикл, каждая итерация которого обработает одну дату
#Перечисляем даты (функция tqdm позволяет примерно оценивать время исполнения программы)
for date in tqdm(date_list):

  #Генерируем из даты ссылку
  page = f"https://panorama.pub/news/{date}"

  #Выкачиваем страницу
  result = requests.get(page)
  html = result.text

  #Парсим html, чтобы извлечь из него нужный фрагмент
  soup = BeautifulSoup(html)

  #Извлекаем и преобразуем в список строк, содержащих новости
  titles = soup.find_all('div', {'class': "pt-2 text-xl lg:text-lg xl:text-base text-center font-semibold"})
  titles = [x.text.strip() for x in titles]

  #Добавляем извлечённые новости к списку заголовков, которые будут потом в табличке с метаинформацией
  all_titles.extend(titles)

  #Добавляем в список и информациями о датах столько дат, сколько у нас заголовков статей
  all_dates.extend([date] * len(titles))

  #Извлекаем ссылки на конкретные новости
  #Для этого нам приходится найти тег со ссылкой на неё. Но оказывается, что ссылка
  #не полная — и мы добавляем в начало адрес основной страницы Панорамы
  links = soup.find_all('a', {'class': "flex flex-col rounded-md hover:text-secondary hover:bg-accent/[.1] mb-2"}, href=True)
  links = [f"https://panorama.pub{link['href']}" for link in links]

  #Добавляем ссылки в список для таблички
  all_links.extend(links)

  #Идём по каждой из извлечённых ссылок
  for link in links:

    #Скачиваем её и парсим
    result = requests.get(link)
    html = result.text
    soup = BeautifulSoup(html)

    #Извлекаем тело статьи
    page_text = soup.find('div', {'itemprop': "articleBody"})

    #Извлекаем только то, что в абзацах
    lines = [line.text.strip() for line in page_text.find_all('p')]

    #Соединяем абзацы в одну строку, убираем неразрывные пробелы и переносы строки
    all_texts.extend([' '.join(lines).replace('\xa0', " ").replace('\n', ' ')])
    sphere = soup.find('a', {'class': "badge"})
    if sphere == None:
      all_spheres.append(sphere)
    else:
      all_spheres.append(sphere.text)

#Это всё выглядит сложно, но суть в том, что необходимо разобраться с тем
#откуда получить ссылки с нужной информацией  — а затем их выкачать и
#извлечь информацию

  0%|          | 0/100 [00:00<?, ?it/s]

In [32]:
#Как выглядит наш текст
all_texts[0]

'По итогам 2023 года правоохранительные органы помогли работодателям вернуть 32408 беглых испольщиков, ещё 10282 человека сейчас числятся беглецами, следует из статистических данных. Возбуждены тысячи дел о бессрочном сыске. В отдельных случаях ОМОН был вынужден прибегнуть к силе. Так, в долине Дона была обнаружена станица, организованная московскими рестораторами, сбежавшими из кофеен и лунжей на Патриарших прудах, не отработав барщину. Атаман Фёдор Чудин попытался оказать организованное сопротивление, устроив дымовую завесу из кальянного дыма и атаковав силовиков зажигательными смесями из текилы и джина. Бунт был жестко подавлен, а его зачинщики получили по 10 лет каторжных работ в столовых РЖД и Росавтодора. Как отмечается в сводке, около 55% всех сбежавших были пойманы на Дону, еще около 30% – в верховьях Яика. В полиции указывают, что беглецы планировали вступить в «организованные преступные группы и разбойничьи шайки», а также примкнуть к различным «бунтовщикам и смутьянам». Пода

In [33]:
#Убедимся, что мы ничего не сломали, и у нас столько же заголовков, сколько дат
len(all_titles) == len(all_dates)

True

## Слишком сложно?..

К счастью, нередко не нужно руками с нуля писать подобные программы для выкачки сайтов.

Некоторые сайты, в том числе крупные соцсети, сами умеют представлять данные в формате, удобном не для пользователя-человека, а для компьютера. Это анзывается API — см., например, [VK API](https://dev.vk.com/api/overview). Но там тоже нужно разбираться...

Кроме этого, есть большое количество готовых модулей или инструментов на основе Python. Универсальная, но достаточно сложная в использовании программа, которая скачивает сайты, самостоятельно переходя по ссылкам, которые находит на их странице (это называется Crawler), — [Scrapy](https://scrapy.org/).

## Экспорт текстов

Осталось сохранить полученные тексты, чтобы не выкачивать каждый раз и иметь доступ в любой момент. Для этого оформим их в виде таблицы, которая содержит всю метаминформацию.

In [34]:
#Создаём индекс, чтобы пронумеровать каждый текст (это нам понадобится позже)

index = [a for a in range(len(all_dates))]

#Добавляем в особую табличку тексты. Для этого подаём функции DataFrame словарь
#В котором на первом месте — название столбца, на втором — содержимое столбца
#Все столбцы должны быть одинаковой длины
corpus = pd.DataFrame({'index': index, 'date': all_dates, 'sphere': all_spheres,
                       'title': all_titles, 'text': all_texts,
                       'link': all_links})

corpus.head()

,index,date,sphere,title,text,link
0,0,28-2-2024,Общество,В 2023 году правоохранительные органы вернули ...,По итогам 2023 года правоохранительные органы ...,https://panorama.pub/news/v-2023-godu-sotrudniki
1,1,28-2-2024,Общество,Интернет-ресурсам придётся ретушировать фотогр...,Постановлением министерства информации с 1 мар...,https://panorama.pub/news/internet-resursy-zas...
2,2,28-2-2024,Экономика,Курировать поставки лечебных трав из Афганиста...,Премьер-министр России Михаил Мишустин поручил...,https://panorama.pub/news/kurirovat-postavki-l...
3,3,28-2-2024,Общество,В Дагестане несовершеннолетнего осудили за вов...,В Дагестане суд вынес уникальный приговор – не...,https://panorama.pub/news/v-dagestane-nesovers...
4,4,28-2-2024,Политика,Юрий Лоза предостерёг Макрона от ввода войск в...,Музыкант и композитор Юрий Лоза предупредил пр...,https://panorama.pub/news/urij-loza-predostere...


In [35]:
#Экспортируем файл. В обычном Питоне он запишется на компьютер ()
corpus.to_csv('panorama_corpus.tsv', sep = '\t', encoding='utf-8',  index = False)